In [1]:
import logging, yaml, glob
from uw.like2 import configuration
from skymaps import Band
from fermipy import gtanalysis
reload(gtanalysis)
os.chdir(os.path.expandvars('$FERMI/skymodels/P305_8years/uw8210'))
os.environ['CUSTOM_IRF_NAMES']='' # it wants this set, but I use CALDB

In [26]:
# set up the pointlike configuration, extract stuff

cfg = configuration.Configuration('.', quiet=True, postpone=True)
os.environ['CALDB']=cfg.caldb
print 'CALDB set to ', os.environ['CALDB']
irf= cfg.irf
roidir = Band(12).dir(100) # choose an arbitrary RoI
diffuse_dir =os.path.expandvars('$FERMI/diffuse/') 
galdiff = diffuse_dir+cfg['diffuse']['ring']['filename']
isodiff = [diffuse_dir+cfg['diffuse']['isotrop']['filename'].replace('**',x) for x in('FRONT','BACK')]            

CALDB set to  /afs/slac/g/glast/groups/canda/irfs/p8_merit/P8R3_V2/CALDB


In [29]:
# set up components, here for front back on a single energy range
ie=0 #first band

order=6 #make depend on ie and evtype
components = [dict(dict(name='{:02d}'.format(evtype+2*ie), #gets replaced
                        selection=dict(evtype=evtype, emin=10**2+ie/4., emax=10**2.25+ie/4.),
                        binning=dict(hpx_order=order), 
                        model=dict(isodiff=isodiff[evtype]))) 
              for evtype in (0,1)]
components

[{'binning': {'hpx_order': 6},
  'model': {'isodiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_FRONT_zmax80_EGBfree.txt'},
  'name': '00',
  'selection': {'emax': 177.82794100389228, 'emin': 100.0, 'evtype': 0}},
 {'binning': {'hpx_order': 6},
  'model': {'isodiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_BACK_zmax80_EGBfree.txt'},
  'name': '01',
  'selection': {'emax': 177.82794100389228, 'emin': 100.0, 'evtype': 1}}]

In [30]:
config = yaml.load("""
data:
    evfile: {ft1}
    scfile: null
    ltcube: {ltcube}
    
binning:
    projtype: 'HPX'
    enumbins: 1
    hpx_order: 6 #64 override in components
    coordsys: 'GAL'

selection:
    ra: {ra}
    dec: {dec}
    #filter: 'ZENITH_ANGLE <= 100'

gtlike:
    irfs: {irf}
    
model:
    galdiff: {galdiff}
    isodiff:
    catalogs:
        - '3FGL'
    sources: []

components: {components}

fileio:
    outdir: 'fermipy'

""".format(ltcube= sorted(glob.glob('/afs/slac/g/glast/groups/catalog/P8_P302/zmax105/ltcube*.fits'))[0],# /cfg.dataset.ltcube, 
           irf=irf, #'P8R2_SOURCE_V6', #
           ft1=sorted(glob.glob('/afs/slac/g/glast/groups/catalog/P8_P305/zmax105/P305*.fits'))[0],
           galdiff=galdiff,
           components=components,
           ra=roidir.ra(),
           dec=roidir.dec(),
           ))
config

{'binning': {'coordsys': 'GAL',
  'enumbins': 1,
  'hpx_order': 6,
  'projtype': 'HPX'},
 'components': [{'binning': {'hpx_order': 6},
   'model': {'isodiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_FRONT_zmax80_EGBfree.txt'},
   'name': '00',
   'selection': {'emax': 177.82794100389228, 'emin': 100.0, 'evtype': 0}},
  {'binning': {'hpx_order': 6},
   'model': {'isodiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffuse/4YP304_P8R2_SOURCE_V6_BACK_zmax80_EGBfree.txt'},
   'name': '01',
   'selection': {'emax': 177.82794100389228, 'emin': 100.0, 'evtype': 1}}],
 'data': {'evfile': '/afs/slac/g/glast/groups/catalog/P8_P305/zmax105/P305_Source_001_zmax105.fits',
  'ltcube': '/afs/slac/g/glast/groups/catalog/P8_P302/zmax105/ltcube_001_zmax105.fits',
  'scfile': None},
 'fileio': {'outdir': 'fermipy'},
 'gtlike': {'irfs': 'P8R3_SOURCE_V2'},
 'model': {'catalogs': ['3FGL'],
  'galdiff': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/diffu

In [31]:
self = gtanalysis.GTAnalysis(config)

2018-03-28 13:36:36 INFO    GTAnalysis.__init__(): 
--------------------------------------------------------------------------------
fermipy version 0.16.0+170.gf71ab.dirty 
ScienceTools version ScienceTools-11-05-00


In [32]:
self.setup() 

2018-03-28 13:36:56 INFO    GTAnalysis.setup(): Running setup.
2018-03-28 13:36:56 INFO    GTBinnedAnalysis.setup(): Running setup for component 00
2018-03-28 13:36:56 INFO    GTBinnedAnalysis._select_data(): Start data selection files: {'ft1': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/ft1_00.fits', 'srcmdl': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/srcmdl_00.xml', 'ccube': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/ccube_00.fits', 'bexpmap': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/bexpmap_00.fits', 'ltcube': '/afs/slac/g/glast/groups/catalog/P8_P302/zmax105/ltcube_001_zmax105.fits', 'ft1_filtered': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/ft1_filtered_00.fits', 'srcmap': '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/srcmap_00.fits', 'bexpmap_roi': '/nfs/farm/g/glast/g/catalog/pointlike

IOError: [Errno 2] No such file or directory: '/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P305_8years/uw8210/fermipy/bexpmap_00.fits'

In [102]:
[(x, self.config[x]) for x in 'data binning selection gtlike fileio components'.split()]

[('data',
  {'cacheft1': True,
   'evfile': '/afs/slac/g/glast/groups/catalog/P8_P305/zmax105/P305_Source_001_zmax105.fits',
   'ltcube': '/afs/slac/g/glast/groups/catalog//pointlike/fermi/data/../../../P8_P305/ltcube_8years_zmax100.fits',
   'scfile': None}),
 ('binning',
  {'binsperdec': 8.0,
   'binsz': 0.1,
   'coordsys': 'GAL',
   'enumbins': None,
   'hpx_ebin': True,
   'hpx_order': 6,
   'hpx_ordering_scheme': 'RING',
   'npix': None,
   'proj': 'AIT',
   'projtype': 'HPX',
   'roiwidth': 10.0}),
 ('selection',
  {'convtype': None,
   'dec': 24.200626132,
   'emax': None,
   'emin': None,
   'evclass': None,
   'evtype': None,
   'filter': None,
   'glat': None,
   'glon': None,
   'logemax': None,
   'logemin': None,
   'phasemax': None,
   'phasemin': None,
   'ra': 162.387974602,
   'radius': None,
   'roicut': 'no',
   'target': None,
   'tmax': None,
   'tmin': None,
   'zmax': None}),
 ('gtlike',
  {'bexpmap': None,
   'bexpmap_base': None,
   'bexpmap_roi': None,
   'bex